In [64]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# 21.10.2024
# 10.11.2024 Исправлена логика подсчёта ошибки и исправлена логика восстановления изображения из блоков
# 21.11.2024 Исправлена логика подсчёта коэффициента сжатия
# 24.11.2024 Исправлена логика подсчёта коэффициента сжатия

In [65]:
import numpy as np
from PIL import Image

In [66]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3
MAIN_FLOAT_TYPE = np.float16

In [67]:
def image_to_blocks(image, b_h, b_w, overlap = 0):
    i_h, i_w = image.shape[:2]

    step_h = int(b_h * (1 - overlap))
    step_w = int(b_w * (1 - overlap))

    blocks = []

    for i in range(0, i_h - b_h + 1, step_h):
        for j in range(0, i_w - b_w + 1, step_w):
            block = image[i:i+b_h, j:j+b_w]                                  
            blocks.append(block)    
    
    if i_h % b_h != 0:
        for j in range(0, i_w - b_w + 1, step_w):
            block = image[i_h-b_h:i_h, j:j+b_w]
            blocks.append(block)    
    
    if i_w % b_w != 0:
        for i in range(0, i_h - b_h + 1, step_h):
            block = image[i:i+b_h, i_w-b_w:i_w]
            blocks.append(block)    
    
    if i_h % b_h != 0 and i_w % b_w != 0:
        block = image[i_h-b_h:i_h, i_w-b_w:i_w]
        blocks.append(block)
    
    return np.asarray(blocks)


def blocks_to_image(image_blocks, image_shape, b_h, b_w, overlap = 0):
    i_h, i_w = image_shape[:2]
    c = image_shape[2] if len(image_shape) == 3 else 1

    restored_image = np.zeros((i_h, i_w, c), dtype=np.float64)
    count_matrix = np.zeros((i_h, i_w), dtype=np.float64)
    
    step_h = int(b_h * (1 - overlap))
    step_w = int(b_w * (1 - overlap))
    
    block_index = 0
    
    for i in range(0, i_h - b_h + 1, step_h):
        for j in range(0, i_w - b_w + 1, step_w):
            block = image_blocks[block_index]            
            restored_image[i:i+b_h, j:j+b_w] += block
            count_matrix[i:i+b_h, j:j+b_w] += 1
            block_index += 1    
    
    if i_h % b_h != 0:
        for j in range(0, i_w - b_w + 1, step_w):
            block = image_blocks[block_index]
            restored_image[i_h-b_h:i_h, j:j+b_w] += block
            count_matrix[i_h-b_h:i_h, j:j+b_w] += 1
            block_index += 1    
    
    if i_w % b_w != 0:
        for i in range(0, i_h - b_h + 1, step_h):
            block = image_blocks[block_index]
            restored_image[i:i+b_h, i_w-b_w:i_w] += block
            count_matrix[i:i+b_h, i_w-b_w:i_w] += 1
            block_index += 1    
    
    if i_h % b_h != 0 and i_w % b_w != 0:
        block = image_blocks[block_index]
        restored_image[i_h-b_h:i_h, i_w-b_w:i_w] += block
        count_matrix[i_h-b_h:i_h, i_w-b_w:i_w] += 1    
    
    count_matrix[count_matrix == 0] = 1    
    restored_image = restored_image / count_matrix[..., np.newaxis]
    restored_image[restored_image > 255] = 255    
    
    return restored_image.astype(np.uint8)

In [68]:
def normalize_weights(weights):
    norms = np.linalg.norm(weights, axis=0)
    return weights / norms

# Функция активации
def linear_activation(x):
    return x

class LRNN:
    def __init__(self, input_dim, latent_dim, learning_rate=0.001):
        np.random.seed(1)

        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.learning_rate = learning_rate        
        
        self.W_enc = (normalize_weights(np.random.rand(self.input_dim, self.latent_dim))
                      .astype(MAIN_FLOAT_TYPE))
        self.W_dec = (normalize_weights(np.random.rand(self.latent_dim, self.input_dim))
                      .astype(MAIN_FLOAT_TYPE))        

        self.epoch: int = 0
    
    def forward(self, x):
        z = linear_activation(x @ self.W_enc)
        x_reconstructed = linear_activation(z @ self.W_dec)
        return z, x_reconstructed
    
    def backward(self, x, x_reconstructed):
        error = x_reconstructed - x        
        
        dW_dec = (x @ self.W_enc).T @ error
        dW_enc = (x.T @ error) @ self.W_dec.T                       
        
        self.W_dec -= self.learning_rate * dW_dec
        self.W_enc -= self.learning_rate * dW_enc        
        
        self.W_dec = normalize_weights(self.W_dec)
        self.W_enc = normalize_weights(self.W_enc)        
    
    def squared_error(self, y_true, y_predicted) -> float:
        error = 0
        y_true, y_predicted = np.array(y_true)[0], np.array(y_predicted)[0]
        if len(y_true) != len(y_predicted):
            raise ValueError('True and predicted vectors must be same size!')
        for i in range(len(y_true)):
            error += (y_true[i] - y_predicted[i]) * (y_true[i] - y_predicted[i])
        return error
    
    def train(self, data, epochs=1000, max_loss: float = 100, learn_by_loss: bool = False):
        for epoch in range(epochs):
            self.epoch += 1
            total_loss = 0
            for x in data:                
                x = np.matrix(x)
                _, x_reconstructed = self.forward(x)
                self.backward(x, x_reconstructed)
            for x in data:
                _, x_reconstructed = self.forward(x)
                total_loss += self.squared_error(x, x_reconstructed)
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss}')                        
            if learn_by_loss and total_loss <= max_loss:
                break            

In [69]:
# Image compression/decompression pipeline
def compress_image(compression_weights, img_array, channels_amount: int,
                   block_height: int, block_width: int, overlap: float = 0):    
    normalized = (2.0 * img_array.astype(MAIN_FLOAT_TYPE) / MAX_RGB_VALUE) - 1.0    
    blocks = image_to_blocks(normalized, block_height, block_width, overlap)
    blocks = blocks.reshape((len(blocks), block_height * block_width, channels_amount))
    if channels_amount == 3:
        blocks = blocks.transpose(0, 2, 1)    
    blocks = np.einsum('ijk,kl->ijl', blocks, compression_weights)     
    return blocks
    

def decompress_image(decompression_weights, compressed_img, img_shape, channels_amount: int,
                     block_height: int, block_width: int, overlap: float = 0) -> Image.Image:
    compressed_img = np.einsum('ijk,kl->ijl', compressed_img, decompression_weights)
    compressed_img = MAX_RGB_VALUE * (compressed_img + 1.0) / 2.0
    if channels_amount == 3:
        compressed_img = compressed_img.transpose(0, 2, 1)
    compressed_img = compressed_img.reshape((len(compressed_img), block_height, block_width, channels_amount))    
    img_array = blocks_to_image(compressed_img, img_shape, block_height, block_width, overlap)    
    return Image.fromarray(img_array).convert('RGB' if channels_amount == 3 else 'L')

In [70]:
# Collecting everything
np.random.seed(1)

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 40 

img = Image.open('mountains.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, n)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])


network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=2500)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)

dimg.save('compression-decompression_test.jpg')

img_diff_array = np.minimum(np.abs(img_array - dimg_array), np.abs(dimg_array - img_array))
img_diff = Image.fromarray(img_diff_array).convert('RGB')
img_diff.save('compression-decompression_test_diff.jpg')


Epoch 1/15000, Loss: 7996.492115596032
Epoch 2/15000, Loss: 5466.891730898056
Epoch 3/15000, Loss: 4889.23423671475
Epoch 4/15000, Loss: 4555.97773912239
Epoch 5/15000, Loss: 4330.3842542823495
Epoch 6/15000, Loss: 4152.424401245242
Epoch 7/15000, Loss: 4017.380836415817
Epoch 8/15000, Loss: 3902.669337599564
Epoch 9/15000, Loss: 3804.2766454414987
Epoch 10/15000, Loss: 3720.6052414951355
Epoch 11/15000, Loss: 3645.7080549723523
Epoch 12/15000, Loss: 3578.5507968547226
Epoch 13/15000, Loss: 3522.0347287703585
Epoch 14/15000, Loss: 3472.198749294733
Epoch 15/15000, Loss: 3426.4397006516015
Epoch 16/15000, Loss: 3382.27502610465
Epoch 17/15000, Loss: 3342.9296719894637
Epoch 18/15000, Loss: 3306.611191750426
Epoch 19/15000, Loss: 3272.5240156645827
Epoch 20/15000, Loss: 3239.7221579029565
Epoch 21/15000, Loss: 3210.0773399920013
Epoch 22/15000, Loss: 3182.3263686397277
Epoch 23/15000, Loss: 3155.5980149245092
Epoch 24/15000, Loss: 3130.7866366020316
Epoch 25/15000, Loss: 3107.47293700888

In [71]:
# Test on another pic
img = Image.open('mountains2.jpg')
img_array = np.asarray(img)
shape = img_array.shape
compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test2.jpg')

img_diff_array = np.minimum(np.abs(img_array - dimg_array), np.abs(dimg_array - img_array))
img_diff = Image.fromarray(img_diff_array).convert('RGB')
img_diff.save('compression-decompression_test2_diff.jpg')